# 📊 Scraped Database Processing & Salary Intelligence

## Project Introduction
This Jupyter Notebook focuses on the processing, cleansing, and enrichment of data extracted from various job boards (Manfred and Tecnoempleo). The primary objective is to decode the current state of the Spanish IT labor market by answering key strategic questions:
* Which roles command the highest compensation in the current market?
* What is the real impact of seniority on salary scaling?
* How has the "100% On-site" model shifted towards Remote and Hybrid work?

## Methodology Overview
Due to the prevailing **salary opacity** in the sector, this project does not merely aggregate data. It implements a **Data Engineering pipeline** that:
1. **Normalizes** disparate data structures from multiple web sources.
2. **Triangulates** missing information using industry-standard benchmarks ([Randstad Research 2026](https://www.randstadresearch.es/tendencias-salariales/?gad_campaignid=23330767618), `Sector de IT & Telecom`) .
3. **Consolidates** a high-integrity dataset ready for Business Intelligence exploitation.

By bridging the gap between raw web-scraped text and structured insights, this analysis provides a transparent view of the IT compensation landscape in 2026.

## 🛠️ Phase 0: Setup and Environment Configuration

This initial section establishes the technical foundation of the project. We import the fundamental libraries required for the entire data lifecycle:
* **Ingestion:** `Playwright` for browser automation and dynamic web data extraction.
* **Processing:** `Pandas` and `NumPy` for data structure manipulation and wrangling.
* **Visualization:** `Seaborn` and `Matplotlib` for preliminary statistical analysis.

The goal is to ensure a scalable and efficient environment for processing the collected job postings.

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import sys

# Importar patrones de regex
sys.path.insert(0, './src')
from src.regex_salarios import REGEX_SALARIOS

## 🧬 Phase 1: Manfred Data Processing

Manfred serves as our primary source of **real-world data**. It is characterized by high transparency, allowing us to capture explicit salary ranges in the majority of its postings.

**Architectural Decision:**
In this project, we opted to generate separate databases for each source, as different websites present information in distinct structures. Consequently, the data processing logic is isolated within this stage rather than during the scraping phase. Manfred provides a concise summary containing all key data points, requiring minimal processing and significantly simplifying data extraction from Natural Language text.

**Actions taken:**
1. Text cleansing and normalization of labels (Seniority levels and Job Roles).
2. Conversion of salary ranges into mean values for statistical analysis.
3. Initial Exploratory Data Analysis (EDA) to validate sample distribution.

This database acts as an "anchor" to understand actual market salaries before proceeding with estimations for other sources.

In [ ]:
# 1. Cargamos los datos brutos que conseguiste
df = pd.read_csv("../data/ofertas_generico.csv")

# 2. Función "limpiadora" (Parsing)
def limpiar_oferta(row):
    texto = row['contenido']
    
    # Regex para sueldo bruto mensual en euros (puede tener formato €2,838K o €2838 o €2.838)
    sueldo_pattern = r'€\s*([0-9,.K-]+?)(?:\s*\||$)'
    sueldo_match = re.search(sueldo_pattern, texto)
    sueldo = 0
    if sueldo_match:
        cantidad_str = sueldo_match.group(1).strip()
        # Extraer el valor medio (ej: "30-50K" -> "40")
        if '-' in cantidad_str:
            partes = cantidad_str.split('-')
            if partes[1].endswith('K'):
                partes[1] = partes[1].replace('K','')
                partes[0] = int(partes[0]) * 1000
                partes[1] = int(partes[1]) * 1000
            cantidad_str = str((int(partes[0]) + int(partes[1])) // 2)
        # Convertir K a miles
        if 'K' in cantidad_str:
            cantidad_str = cantidad_str.replace('K', '').replace(',', '').replace('.', '')
            cantidad = str(int(float(cantidad_str)) * 1000)
        else:
            cantidad = cantidad_str.replace(',', '').replace('.', '')
        sueldo = int(cantidad)
    
    # Ubicación
    ubicacion_pattern = r'UBICACIÓN\s*\|\s*([^\|]+)'
    ubicacion_match = re.search(ubicacion_pattern, texto)
    ubicacion = ubicacion_match.group(1).strip() if ubicacion_match else "N/A"

    # Modalidad de trabajo - más robusta
    modalidad = "N/A"
    # Primero buscamos el patrón "TELETRABAJO | porcentaje"
    modalidad_pattern = r'TELETRABAJO\s*\|\s*(\d+)%'
    modalidad_match = re.search(modalidad_pattern, texto)
    if modalidad_match:
        porcentaje = int(modalidad_match.group(1))
        if porcentaje == 100:
            modalidad = "Telemático"
        elif porcentaje == 0:
            modalidad = "Presencial"
        else:
            modalidad = "Híbrido"
    else:
        # Si no encontramos TELETRABAJO, buscamos palabras clave
        if re.search(r'Híbrido', texto, re.IGNORECASE):
            modalidad = "Híbrido"
        elif re.search(r'Presencial', texto, re.IGNORECASE):
            modalidad = "Presencial"
        elif re.search(r'Telemático|Remoto|100% remoto', texto, re.IGNORECASE):
            modalidad = "Telemático"
    
    # Generamos un array, por el pipe "|" que pusimos en el scraper
    partes = [p.strip() for p in texto.split('|')]
    
    return pd.Series({
        'Puesto': partes[2] if len(partes) > 0 else "N/A",
        # 'Empresa': partes[1] if len(partes) > 1 else "N/A",
        'Sueldo_Bruto': sueldo if sueldo != 0 else "N/A",
        'Ubicación': ubicacion,
        'Modalidad_Trabajo': modalidad,
        'URL': row['url'],
        'Fecha_Captura': row['fecha_captura']
    })

def categorizar_rol(texto):
    texto = str(texto).lower()
    if 'analyst' in texto or 'analista' in texto: return 'Data Analyst'
    if 'engineer' in texto or 'ingeniero' in texto: return 'Data Engineer'
    if 'science' in texto or 'científico' in texto: return 'Data Scientist'
    if 'backend' in texto: return 'Backend/Fullstack Developer'
    if 'fullstack' in texto: return 'Backend/Fullstack Developer'
    return 'IT Generalist'

def categorizar_nivel(texto):
    texto = str(texto).lower()
    if any(w in texto for w in ['senior', 'sr', 'lead', 'principal', 'cio']): return 'Senior'
    if any(w in texto for w in ['junior', 'jr', 'entry', 'trainee']): return 'Junior'
    return 'Mid'

# 3. Aplicamos la limpieza a todo el DataFrame
df_manfred = df.apply(limpiar_oferta, axis=1)
# Si es Telemático, rellenamos la ubicación con 'Remoto' para que el gráfico sea más limpio
df_manfred.loc[df_manfred['Modalidad_Trabajo'] == 'Telemático', 'Ubicación'] = 'Remoto'
df_manfred['Rol'] = df_manfred['Puesto'].apply(categorizar_rol)
df_manfred['Nivel'] = df_manfred['Puesto'].apply(categorizar_nivel)
df_manfred['Fuente'] = 'Manfred'

# 4. Ver el resultado (esto en VS Code se ve como una tabla de Excel preciosa)
df_manfred.info()
df_manfred = df_manfred.sort_values('Sueldo_Bruto', ascending=True)
df_manfred.head()

### ⚠️ Sample Limitations and Data Integrity
The processed Manfred dataset is now clean, with no null or empty values. However, we face a critical challenge: the sample consists of only **33 job postings**. 

Any conclusions drawn at this stage lack sufficient statistical power to be considered definitive. Furthermore, most classes are **unbalanced**, which could introduce significant bias. To draw robust, data-driven conclusions, we must expand our dataset by integrating further sources.

In [ ]:
# Configuramos el estilo
plt.figure(figsize=(10, 6))
sns.set_theme(style="whitegrid")

# Ordenamos por sueldo para que el gráfico sea legible
df_sorted = df_manfred.sort_values('Sueldo_Bruto', ascending=False)

# Creamos el gráfico
ax = sns.barplot(x='Sueldo_Bruto', y='Puesto', data=df_sorted, palette='viridis')

# Añadimos etiquetas de datos
plt.title('Top Ofertas por Salario Bruto (Manfred)', fontsize=15)
plt.xlabel('Salario Bruto Anual (€)', fontsize=12)
plt.ylabel('Posición', fontsize=12)

plt.show()

### 📊 Work Modality vs. Salary
In an attempt to extract early insights, we calculate the correlation between work modality and salary. The core question: *Do on-site roles offer higher compensation in the current market?*

In [ ]:
# 1. Agrupamos por modalidad y sacamos estadísticas básicas
stats_modalidad = df_manfred.groupby('Modalidad_Trabajo')['Sueldo_Bruto'].agg(['mean', 'median', 'count', 'min', 'max'])
print("Estadísticas de Sueldo por Modalidad:")
print(stats_modalidad)

# 2. Visualización: Boxplot (Distribución de sueldos)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Modalidad_Trabajo', y='Sueldo_Bruto', data=df_manfred, palette="Set2")
plt.title('Distribución de Sueldos por Modalidad de Trabajo', fontsize=15)
plt.ylabel('Sueldo Bruto Anual (€)')
plt.show()

#### Preliminary Findings:
Given the sample size (23 Hybrid vs. 9 Remote/Teleworking), the classes are unbalanced, and any immediate conclusions may be skewed. However, based on this specific sample of 33 postings, **Remote roles tend to command higher salaries** than their Hybrid counterparts.

## 📈 Phase 2: Tecnoempleo Processing and Market Benchmarking

Tecnoempleo provides the largest volume of job postings but presents a critical challenge: **salary opacity**. To address this data gap without biasing the analysis, we apply a **data triangulation** technique.

---
<p align="center">
  <img src="../03_Enrichment/Captura%20Benchmark%20Randstad.png" alt="Randstad Salary Trends 2026">
  <br>
  <sub><b>Figure 1:</b> Reference salary ranges for the IT sector in Spain (Source: Randstad Research 2026).</sub>
</p>

---

**Benchmarking Methodology:**
To impute missing salaries, we use the **2026 Salary Trends Report from Randstad Research** as a reference. This report provides average salary ranges segmented by profile and experience within the Spanish IT & Telecom sector.

**Market Evidence:**
As shown in the Randstad data, profiles such as **Data Analysts (€23k - €57k)** or **Developers (€22k - €81k)** exhibit massive seniority gaps. This external evidence validates our assignment logic and allows us to complete the dataset in a manner consistent with the current socio-economic reality.

In [ ]:
# 1. Cargar el dataset de tecnoempleo
df_tecno = pd.read_csv("../data/ofertas_tecnoempleo_raw.csv")

# Convertir a minúsculas una vez
df_tecno['contenido'] = df_tecno['contenido'].apply(str.lower)

# Lista de ciudades españolas principales
CIUDADES_ESPAÑA = {
  'madrid', 'barcelona', 'valencia', 'sevilla', 'bilbao', 'málaga', 'murcia',
  'palma', 'las palmas', 'córdoba', 'valladolid', 'vigo', 'gijón', 'eibar',
  'albacete', 'alicante', 'almería', 'ávila', 'badajoz', 'burgos', 'cádiz',
  'castellón', 'ciudad real', 'cuenca', 'girona', 'guadalajara', 'huelva',
  'huesca', 'jaén', 'león', 'lérida', 'logroño', 'lugo', 'navarra', 'orense',
  'oviedo', 'palencia', 'pamplona', 'pontevedra', 'salamanca', 'santander',
  'segovia', 'soria', 'tarragona', 'teruel', 'toledo', 'tortosa', 'zamora',
  'zaragoza', 'tenerife', 'ibiza', 'mallorca', 'menorca', 'ceuta', 'melilla',
  'alcobendas', 'leganés', 'getafe', 'móstoles', 'torrejón', 'alcalá'
}

# --- MENTALIDAD SPARK: Definimos funciones puras de transformación ---
def extract_salary(text):
  if pd.isna(text): 
    return None
  salaries = []
  for pattern_dict in REGEX_SALARIOS:
    patron = pattern_dict['patron']
    matches = re.findall(patron, text, re.IGNORECASE)
    for m in matches:
      # m puede ser string o tuple (si hay grupos de captura)
      if isinstance(m, tuple):
        m = m[0] if m[0] else m[1] if len(m) > 1 else None
      
      if m:
        # Normalizar
        num_str = str(m).replace('.', '').replace(',', '').replace('K', '').replace('k', '')
        if num_str.isdigit():
          val = int(num_str)
          if val < 10000:  # Si < 10k, probablemente está en K
            val = val * 1000
          if 10000 <= val <= 500000:
            salaries.append(val)
    
  return max(salaries) if salaries else None

def extract_location(text):
  # SOLO buscar patrones muy específicos y fiables
  location_patterns = [
    r'(?:ubicación|localización|location)[:\s]+([a-záéíóúñ\s]+?)(?:,|\.|\||$)',  # ubicación: Madrid
    r'\(([a-záéíóúñ\s]+)\)',  # (Madrid)
    r'(?:en|based in|based at)\s+([a-záéíóúñ]+)(?:\s|,|\.)',  # en Madrid o en Barcelona
  ]
  
  for pattern in location_patterns:
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      loc = match.group(1).strip()
      loc_lower = loc.lower()
      # Solo retorna si está en lista de ciudades conocidas
      if loc_lower in CIUDADES_ESPAÑA:
        return loc.capitalize()
  
  return "No especificada"

def extract_modality(text):
  if any(word in text for word in ['remoto', 'telemático', 'teletrabajo', 'home office', '100% remoto', 'remote']):
    return 'Remoto'
  if any(word in text for word in ['híbrido', 'presencial', 'oficina', 'on-site']):
    return 'Híbrido/Presencial'
  return 'No especificado'


# Usamos .apply() en lugar de list comprehensions con zip para evitar las tuplas
df_tecno['Sueldo_Bruto'] = df_tecno['contenido'].apply(extract_salary)
df_tecno['Ubicación'] = df_tecno['contenido'].apply(extract_location)
df_tecno['Modalidad_Trabajo'] = df_tecno['contenido'].apply(extract_modality)

# Estas ya las tenías bien enfocadas
df_tecno['Rol'] = df_tecno['Puesto'].apply(categorizar_rol)
df_tecno['Nivel'] = (df_tecno['Puesto'] + " " + df_tecno['contenido']).apply(categorizar_nivel)

df_tecno['Sueldo_Bruto'] = pd.to_numeric(df_tecno['Sueldo_Bruto'], errors='coerce')
df_tecno_ready = df_tecno[['Puesto', 'Sueldo_Bruto', 'Ubicación', 'Modalidad_Trabajo', 'Rol', 'Nivel', 'Fuente', 'URL', 'Fecha_Captura']]

df_tecno_ready.head()

## 🔗 Phase 3: Data Integration and Final Consolidation

In this final stage, we merge both sources (`Manfred` + `Tecnoempleo`) into a single master dataset structured for Power BI.

**Data Integrity Criteria:**
We have implemented a logic focused on the **prevalence of real data**. In any record where an explicit salary was provided by the original source, it takes absolute priority over the benchmark estimation. This ensures the final analysis remains as faithful as possible to the original job postings, using estimation only as a tool to prevent loss of representativeness in the total volume.

The result is a master dataset of **119 records**, fully optimized for visual and strategic exploitation.

### 🛠️ Creating the Hays Reference Benchmark
Unlike job boards, Hays bases its data on actual placements rather than just active ads. To account for this, we’ve built a secondary reference dataframe that maps roles and seniority levels across Spain’s main tech hubs using the Hays 2026 guide. This second data source adds an extra layer of validation, making our salary benchmark much more reliable.

In [ ]:
# Seleccionamos únicamente las columnas necesarias
df_manfred = df_manfred[['Puesto', 'Sueldo_Bruto', 'Ubicación', 'Modalidad_Trabajo', 'Rol', 'Nivel', 'Fuente', 'URL', 'Fecha_Captura']]
df_total = pd.concat([df_manfred, df_tecno_ready], ignore_index=True)

df_benchmark_randstad = pd.read_csv("../03_Enrichment/benchmark_sueldo_medio_Randstad.csv")
df_benchmark_hays = pd.read_csv("../03_Enrichment/benchmark_sueldo_medio_Hays.csv")

df_final = pd.merge(
    df_total, 
    df_benchmark_randstad[['Rol', 'Nivel', 'Sueldo_Aprox_Mercado']], 
    on=['Rol', 'Nivel'], 
    how='left'
).rename(columns={'Sueldo_Aprox_Mercado': 'Sueldo_Randstad'})

df_final = pd.merge(
    df_final, 
    df_benchmark_hays[['Rol', 'Nivel', 'Sueldo_Aprox_Mercado']], 
    on=['Rol', 'Nivel'],
    how='left'
).rename(columns={'Sueldo_Aprox_Mercado': 'Sueldo_Hays'})

df_final['Sueldo_Aprox_Mercado'] = df_final[['Sueldo_Randstad', 'Sueldo_Hays']].mean(axis=1)

# Limpieza: Borramos las columnas temporales y la 'Provincia' sobrante del merge
df_final.drop(columns=['Sueldo_Randstad', 'Sueldo_Hays'], inplace=True)
df_final['Sueldo_Final'] = df_final['Sueldo_Bruto'].fillna(df_final['Sueldo_Aprox_Mercado'])
df_final['Tipo_Dato'] = np.where(df_final['Sueldo_Bruto'].notna(), 'Real', 'Estimado')

df_final.to_csv("../data/dataset_final.csv", index=False, encoding='utf-8-sig')

print(f"✅ Proceso terminado. Tenemos {len(df_final)} ofertas listas.")
print(f"📊 Reales: {len(df_final[df_final['Tipo_Dato'] == 'Real'])} | Estimadas: {len(df_final[df_final['Tipo_Dato'] == 'Estimado'])}")

df_final[df_final['Fuente'] == 'Tecnoempleo'].head()

## 💡 Final Strategic Insight: The Seniority Premium

The analysis identifies a **"Seniority Premium" of 119,5%**, confirming that the market value of a Senior profile is double that of a Junior. 

This extreme salary gap, combined with the near-absence of entry-level vacancies (**n=1 out of 119 total offers**), provides clear evidence of a **high-barrier market**. Companies are willing to assume a 100% cost premium to acquire immediate autonomy rather than investing in the development of emerging talent.

---
> **Methodological Note: Data Ensemble & Triangulation**
> 
> To solve the lack of salary transparency, we moved from a single-source estimation to a **Multi-source Ensemble Model**. 
> 
> By averaging **Randstad Research** (comprehensive national data) and **Hays** (specialized hub data), we achieve:
> 1. **Bias Reduction:** We don't rely on the criteria of a single consultant, verifying the data we obtained.
> 2. **Geographic Precision:** We adjust the estimation in high-cost cities where Hays provides deeper granularity.
> 3. **Statistical Robustness:** The "Ground Truth" used for our final Seniority Gap analysis (119,5%) is now backed by two of the most reputable sources in the Spanish labor market.
>